<a href="https://colab.research.google.com/github/Enrico-Call/RL-AKI/blob/data-prep-enrico/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logo_amds.png?raw=1" alt="Logo" width=128px/>

# VUmc Research Project - Reinforcement Learning for Sepsis Prevention
# Data Preparation

AmsterdamUMCdb version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## 1. Clustering

In [ ]:
Sum_of_squared_distances = []
K = range(2,500, 5)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(space[['Kalium (bloed)', 'ABP gemiddeld', 'Kreatinine (bloed)', 'Natrium (bloed)', 'UrineCAD', 'UrineSupraPubis', 'UrineSpontaan',
 'UrineUP',
 'Kreatinine',
 'Nefrodrain re Uit',
 'Nefrodrain li Uit',
 'UrineIncontinentie',
 'gender_Vrouw',
 'agegroup',
 'AKI']])
    Sum_of_squared_distances.append(km.inertia_)

plt.plot(K, Sum_of_squared_distances, 'b-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

def k_means_over_instances(dataset, cols, k, max_iters, n_inits):

        # Take the appropriate columns.
        temp_dataset = dataset[cols]
        # Now apply the k-means algorithm
        kmeans = KMeans(n_clusters=k, max_iter=max_iters, n_init=n_inits, random_state=0).fit(temp_dataset)
        # Add the labels to the dataset
        dataset['cluster'] = kmeans.labels_
        # Compute the solhouette and add it as well.
        silhouette_avg = silhouette_score(temp_dataset, kmeans.labels_)
        silhouette_per_inst = silhouette_samples(temp_dataset, kmeans.labels_)
        dataset['silhouette'] = silhouette_per_inst

        return dataset, silhouette_avg

In [ ]:
# Use k=50 based on previous runs

new_d, sil = k_means_over_instances(space, ['Kalium (bloed)', 'ABP gemiddeld', 'Kreatinine (bloed)', 'Natrium (bloed)', 'UrineCAD', 'UrineSupraPubis', 'UrineSpontaan',
 'UrineUP',
 'Kreatinine',
 'Nefrodrain re Uit',
 'Nefrodrain li Uit',
 'UrineIncontinentie',
 'gender_Vrouw',
 'agegroup',
 'AKI'], 50, 20, 10)

## 2. Bin Values

In [ ]:
# Binning Values

binsv = [-np.inf, 0, new_d['Noradrenaline (Norepinefrine)'].median(), np.inf]
binsf = [-np.inf, 250, new_d['NaCl 0,45%/Glucose 2,5%'].median(), np.inf]
labels = [0, 1, 2]
new_d['vasop'] = pd.cut(new_d['Noradrenaline (Norepinefrine)'], bins=binsv, labels=labels)
new_d['fluid'] = pd.cut(new_d['NaCl 0,45%/Glucose 2,5%'], bins=binsf, labels=labels)

In [ ]:
# 0 = no vasop, no fluid
# 1 = no vasop, low fluid
# 2 = no vasop, high fluid
# 3 = low vasop, no fluid
# 4 = low vasop, low fluid
# 5 = low vasop, high fluid
# 6 = high vasop, no fluid
# 7 = high vasop, low fluid
# 8 = high vasop, high fluid

act = []

for v, f in zip(new_d['vasop'], new_d['fluid']): 
  if v == 0 and f == 0: act.append('0')
  elif v == 0 and f == 1: act.append('1')
  elif v == 0 and f == 2: act.append('2')
  elif v == 1 and f == 0: act.append('3')
  elif v == 1 and f == 1: act.append('4')
  elif v == 1 and f == 2: act.append('5')
  elif v == 2 and f == 0: act.append('6')
  elif v == 2 and f == 1: act.append('7')
  elif v == 2 and f == 2: act.append('8')

new_d['action'] = act

new_d['reward'] = -new_d['AKI']

new_d['next'] = new_d['cluster'].shift(-1)

final = new_d.dropna()
